In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn import init
from torch.nn import Parameter
import json
import numpy as np
import os
import random
import time
import math


In [ ]:
with open('./curve_projection_dataset/metadata.json', 'r') as f:
    metadata = json.load(f)

curve_data = metadata['curve_data']


from matplotlib.path import Path
from torch.utils.data import Dataset
from torchvision import transforms
from PIL import Image
import torch


class CurveDataset(Dataset):
    def __init__(self, metadata, root_dir, transform=None):
        self.root_dir = Path(root_dir)
        self.metadata = metadata["curves"]
        self.transform = transform or transforms.Compose(
            [
                transforms.Resize((224, 224)),
                transforms.ToTensor(),
            ]
        )

    def __len__(self):
        return len(self.metadata)

    def __getitem__(self, idx):
        item = self.metadata[idx]
        image_path = self.root_dir / item["image_path"]
        image = Image.open(image_path).convert("RGB")
        image = self.transform(image)

        coeffs = item["coefficients"]
        label = torch.tensor(list(coeffs.values()), dtype=torch.float32)

        return image, label
    

In [ ]:
dataset = CurveDataset(metadata, "./curve_projection_dataset")
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=32, shuffle=False)

In [ ]:
class hyperParams:
    num_epochs = 100
    bs = 16
    
        

In [ ]:
class CNNModel(nn.Module):
    def __init__(self, num_classes=5):
        super(CNNModel, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, padding=1)
        
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.fc1 = nn.Linear(32 * 56 * 56, 128)
        self.fc2 = nn.Linear(128, num_classes)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.5)
    
    def forward(self, x):
        x = self.pool(self.relu(self.conv1(x)))
        x = self.pool(self.relu(self.conv2(x)))
        x = x.view(-1, 32 * 56 * 56)
        x = self.dropout(self.relu(self.fc1(x)))
        x = self.fc2(x)
        return x

In [ ]:
num_epochs = hyperParams.epochs
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        